In [83]:
import pandas as pd
import cobra
import numpy as np
import matplotlib.pyplot as plt
import json 
from copy import deepcopy

from micom import Community

def save_dict(data, name):
    with open(name, 'w' ) as file:
        json.dump( data, file )

In [84]:
model_DP = cobra.io.read_sbml_model("models/consistent_DP_SNM.xml")
model_SA = cobra.io.read_sbml_model("models/consistent_iYS854_SNM.xml")
print("Growth: ", model_DP.slim_optimize())
print("Growth: ", model_SA.slim_optimize())

Growth:  0.2823653925322476
Growth:  2.558694612613393


In [85]:
snm3 = pd.read_csv("SNM3.csv", sep =";")
snm3.head()

,Compound,BiGG,ModelSeed,KEGG
0,Alanine,ala__L,cpd00035,C00041
1,Arginine,arg__L,cpd00051,C00062
2,Cysteine,cys__L,cpd00084,C00097
3,Glutamic acid,glu__L,cpd00023,C00025
4,Glycine,gly,cpd00033,C00037


In [86]:
import json
compm_SA = json.loads(open("compm_SA.json").read())
compm_DP = json.loads(open("compm_DP.json").read())

In [87]:
compm = dict(list(compm_SA.items()) + list(compm_DP.items()))

In [88]:
community_compm = dict()
for key, val in compm.items():
    key = key[:-1]+ "m"
    community_compm[key] = val

In [89]:
row1 = ["SA", "Staphylococcus", "Staphylococcus aureus", len(model_SA.reactions),len(model_SA.metabolites), "models/consistent_iYS854_SNM.xml"]
row2 = ["DP", "Dolosigranulum", "Dolosigranulum pigrum", len(model_DP.reactions),len(model_DP.metabolites), "models/consistent_DP_SNM.xml" ]
taxonomy = pd.DataFrame(dict(zip(['id', 'genus', 'species', 'reactions', 'metabolites', 'file'],list(zip(row1,row2)))))
com = Community(taxonomy)

In [90]:
reactions_DP = [rec for rec in com.reactions if "__DP" in rec.id]
for rec in reactions_DP:
    id = rec.id[:-4]
    org_rec = model_DP.reactions.get_by_id(id)
    rec.lower_bound = org_rec.lower_bound 
    rec.upper_bound = org_rec.upper_bound

In [91]:
reactions_SA = [rec for rec in com.reactions if "__SA" in rec.id]
for rec in reactions_SA:
    id = rec.id[:-4]
    org_rec = model_SA.reactions.get_by_id(id)
    rec.lower_bound = org_rec.lower_bound 
    rec.upper_bound = org_rec.upper_bound

In [92]:
#com.medium=community_compm

In [93]:
# Turn on shuttels
#sh_DP = [ex for ex in reactions_DP if "EX_" in ex.id]
#sh_SA = [ex for ex in reactions_SA if "EX_" in ex.id]
#for sh in sh_DP: 
 #   sh.lower_bound=-10
#for sh in sh_SA:
 #   sh.lower_bound=-10

In [94]:
#sol = com.optimize()
#MBR_micom_shuttel = sol.growth_rate
#sol

In [118]:
# Custum optimization

def optimize_coopm_community(model, MBR, biomass_reactions):
    #model = model.copy()
    minMBR = 0.1*MBR
    medium = list(model.medium.keys())
    biomass = []
    for id in biomass_reactions:
        biomass.append(model.reactions.get_by_id(id).flux_expression)
    # Binary variables: Theta_i 
    thetas = []
    for i in range(len(medium)):
        thetas.append(model.problem.Variable('theta_'+str(i), type="binary"))

    # Constraints for exchanges, which are turned of for theta_i = 1
    theta_constraints = []
    for i,id in enumerate(medium):
        reaction = model.reactions.get_by_id(id)
        min_bound = model.reactions.get_by_id(id).lower_bound
        cons = model.problem.Constraint(
            (reaction.flux_expression + min_bound*thetas[i]),
            lb=min_bound,
            ub=1000)
        theta_constraints.append(cons)

    # Constraints for growth rates, which must be at least 10% MBR
    constraint_growth = []
    for growth in biomass:
        constraint_growth += [model.problem.Constraint(
        growth,
        lb=minMBR,
        ub=1000)]

    # Adding new variables and constraints.
    model.add_cons_vars(thetas)
    model.add_cons_vars(theta_constraints)
    model.add_cons_vars(constraint_growth)
    

    # Objevtive is maximising turned of exchanges, that is sum of theta_is
    objective = model.problem.Objective(sum(thetas), direction="max")
    model.objective = objective
    model.solver.update()

    return model, thetas

In [119]:
#com.medium = community_compm
model, thetas = optimize_coopm_community(deepcopy(com), 2.55, ["Growth__DP", "BIOMASS_iYS_wild_type__SA"])

In [120]:
model.optimize()

,abundance,growth_rate,reactions,metabolites
compartments,,,,
DP,0.5,0.255,1152,827
SA,0.5,0.255,928,729
medium,NaN,NaN,111,111


In [121]:
medium = list(model.medium.keys())
for i,theta in enumerate(thetas):
    print(medium[i], "Theta: ", theta.primal)

EX_h2o_m Theta:  1.0
EX_arg__L_m Theta:  1.0
EX_his__L_m Theta:  1.0
EX_ile__L_m Theta:  1.0
EX_k_m Theta:  0.0
EX_leu__L_m Theta:  1.0
EX_lys__L_m Theta:  1.0
EX_cl_m Theta:  0.0
EX_co2_m Theta:  1.0
EX_cys__L_m Theta:  0.0
EX_met__L_m Theta:  1.0
EX_mg2_m Theta:  0.0
EX_mn2_m Theta:  0.0
EX_mobd_m Theta:  1.0
EX_na1_m Theta:  0.0
EX_fe2_m Theta:  0.0
EX_nac_m Theta:  0.0
EX_o2_m Theta:  0.0
EX_orn_m Theta:  1.0
EX_phe__L_m Theta:  1.0
EX_pi_m Theta:  1.0
EX_glc__D_m Theta:  0.0
EX_pyr_m Theta:  1.0
EX_glu__L_m Theta:  1.0
EX_ribflv_m Theta:  1.0
EX_ser__L_m Theta:  1.0
EX_so4_m Theta:  0.0
EX_trp__L_m Theta:  1.0
EX_thm_m Theta:  0.9999915600426832
EX_urea_m Theta:  1.0
EX_val__L_m Theta:  1.0
EX_zn2_m Theta:  0.0
EX_4abz_m Theta:  0.9999917396280494
EX_ala__L_m Theta:  1.0
EX_gly_m Theta:  1.0
EX_thr__L_m Theta:  0.0
EX_pro__L_m Theta:  0.0
EX_ca2_m Theta:  0.0
EX_cobalt2_m Theta:  0.0
EX_succ_m Theta:  1.0
EX_cu2_m Theta:  0.0
EX_fum_m Theta:  1.0
EX_ni2_m Theta:  0.0
EX_26dap__M_m

In [122]:
# Creating COOPM medium
sol = model.optimize()
COOPM_COM = dict()
for rec in model.exchanges:
    if rec.flux < 0:
        COOPM_COM[rec.id] = abs(rec.flux)

In [107]:
COOPM_COM

{'EX_k_m': 0.03046924864233228,
 'EX_cl_m': 0.008123693767523203,
 'EX_cys__L_m': 5.041992584664483,
 'EX_mg2_m': 0.011981078730199542,
 'EX_mn2_m': 0.00491888650106331,
 'EX_mobd_m': 8.925e-07,
 'EX_na1_m': 0.26521530208490557,
 'EX_fe2_m': 0.011394582595129366,
 'EX_nac_m': 0.0015292014169234823,
 'EX_o2_m': 9.668257330644145,
 'EX_glc__D_m': 10.0,
 'EX_so4_m': 0.003291188992063504,
 'EX_thm_m': 8.439957316847587e-05,
 'EX_zn2_m': 0.004102295491293246,
 'EX_4abz_m': 8.260371950542626e-05,
 'EX_thr__L_m': 1.000471849561039,
 'EX_pro__L_m': 0.027294417362141003,
 'EX_ca2_m': 0.006627700983082984,
 'EX_cobalt2_m': 0.004507221735795785,
 'EX_cu2_m': 0.004180042803638314,
 'EX_ni2_m': 0.00452563661331598,
 'EX_26dap__M_m': 0.012347342228016093}

In [108]:
with com as model:
    model.medium = COOPM_COM 
    model.optimize()

In [46]:
# Creating COOPM medium
COOPM_COM = dict()
for i,rec in enumerate(model.medium.keys()):
    if thetas[i].primal < 1:
        rec = model.reactions.get_by_id(rec)
        COOPM_COM[rec.id] = abs(rec.flux)

In [47]:
COOPM_COM

{'EX_k_m': 0.030643918573729806,
 'EX_cl_m': 0.008316323372317028,
 'EX_cys__L_m': 5.042481429669955,
 'EX_mg2_m': 0.012262061959113573,
 'EX_mn2_m': 0.0050431955637814906,
 'EX_na1_m': 0.2655123601841713,
 'EX_fe2_m': 0.011639163036664167,
 'EX_nac_m': 0.0015682405958260212,
 'EX_o2_m': 5.468903440353209,
 'EX_glc__D_m': 10.0,
 'EX_so4_m': 0.0033622811484152523,
 'EX_thm_m': 8.510749519527737e-05,
 'EX_zn2_m': 0.0042067046303835166,
 'EX_4abz_m': 8.47274855857808e-05,
 'EX_thr__L_m': 0.06937020546125328,
 'EX_pro__L_m': 0.027996164912055088,
 'EX_ca2_m': 0.006798101137251243,
 'EX_cobalt2_m': 0.004623103740824453,
 'EX_cu2_m': 0.004287512941471624,
 'EX_ni2_m': 0.004641992070296756,
 'EX_26dap__M_m': 0.012664796051686258}

In [24]:
model.optimize()

,abundance,growth_rate,reactions,metabolites
compartments,,,,
DP,0.5,0.261556,1152,827
SA,0.5,0.255000,928,729
medium,NaN,NaN,111,111
